# Momento de Retroalimentación : Reto Datos

---



*   Jorge Chávez Badillo - A01749448
*   Amy Murakami Tsutsumi - A01750185
*   Ariadna Jocelyn Guzmán Jiménez - A01749373
*   Juan Carlos Varela Tellez - A01367002
*   Alan Eduardo Aquino Rosas - A01366912








In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Herramientas y tecnologías utillizadas para el tratamiento de datos

* **[Apache Spark y PySpark](https://spark.apache.org/):** Apache Spark es un motor multi-lenguaje para poder leer y procesar grandes cantidades de datos que motores normales no podrian. Nos ayudará a el procesamiento de grandes cantidades de datos.

* **[Python](https://www.python.org/):** Lenguaje de programación que nos permitirá el uso de múltiples bibliotecas para el análisis de datos y computación científica de Big Data.

* **[SCINCE](https://gaia.inegi.org.mx/scince2020/):** El Sistema para la Consulta de Informacion Censal es una plataforma que nos permitira descargar los datos que necesitamos. Será nuestra base de consulta de datos del INEGI.

* **[Google Colab](https://research.google.com/colaboratory/faq.html):** Servicio en la nube que nos permitirá la ejecución y realización de nuestro código.

* **[Java](https://www.java.com/en/download/help/whatis_java.html):** Nos dará utilidad para la creación de procesos en Big Data, así como aplicaciones.

# Modelo de almacenamiento de datos


Ya realizado el tutorial #1 de Geo Big Data, logramos descargar los datos del Censo de Población y Vivienda 2020 en su distribución SCINCE y los datos del Directorio Estadístico Nacional de Unidades Económicas DENUE en nuestra máquina local. Sin embargo, para nuestro trabajo en equipo y una facilidad en acceso y manupulación a los datos, utilizaremos un almacenamiento en la nube, donde con ayuda de Google Drive, importaremos los datos a nuestros scripts implementados para poder realizar los procedimientos necesarios de nuestro proyecto. 



# Extracción, limpieza y carga de datos

In [ ]:
#  TPOT instalation
!pip install tpot

!pip install -c pip-forge tpot # Core dependencies

!pip install -c pip-forge tpot xgboost dask dask-ml scikit-mdr skrebate # Aditional dependencies


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 1.8 MB/s 
     |████████████████████████████████| 139 kB 23.7 MB/s 
     |████████████████████████████████| 255.9 MB 32 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11953 sha256=cb8eaca103f42860e150a17cae4ee98b1f3d8c5de9eef04cf48c31a940352754
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pip-forge'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pip-forge'


In [ ]:
SPARK_JARS = ["com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.26.0",
              "com.johnsnowlabs.nlp:spark-nlp_2.12:4.1.0"]

def get_spark(master="local[*]", name="Colab"):
    builder = SparkSession.builder.appName(name)
    builder.config('spark.ui.port', '4050')
    builder.config('spark.jars.packages', ",".join(SPARK_JARS))
    builder.config("spark.driver.memory", "16G")
    builder.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    builder.config("spark.kryoserializer.buffer.max", "2000M")
    builder.config("spark.driver.maxResultSize", "0")
    return builder.getOrCreate()

# Spark
# Install pyspark
!pip install pyspark

# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = get_spark()

# Check Spark Session Information
spark


# Findspark
!pip install -q findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 46.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ac1a41ab9c9a1de3f68dc8ae104e785786ea9dd6d5f4351a37aecc5f3268510a
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
# Geopandas
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.9 MB/s 
     |████████████████████████████████| 6.3 MB 24.5 MB/s 
     |████████████████████████████████| 16.7 MB 501 kB/s 


In [ ]:
# Sedona
!pip install apache-sedona

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 3.7 MB/s 


In [ ]:
# Libraries
import findspark
findspark.init()
import shapely
import geopandas as gpd
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from  pyspark.sql.functions import rand
from pyspark.sql.functions import lower,col,lit
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from  pyspark.sql.functions import rand
from pyspark.sql.functions import lower,col,lit
from pyspark.sql.functions import * 

In [ ]:
# Drive connection
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Start Apache Spark
spark = SparkSession. \
builder. \
appName('appName'). \
config("spark.serializer", KryoSerializer.getName). \
config("spark.executor.memory", "5g"). \
config("spark.driver.memory", "10g"). \
config('spark.driver.maxResultSize', '5g'). \
config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.2.0-incubating,org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
getOrCreate()



In [ ]:
# Read data

BD_DENUE = spark.read.parquet(f"/content/gdrive/Shareddrives/Reto INEGI/Data_INEGI/DENUE_Parquets/*.parquet")
BD_DENUE.createOrReplaceTempView("denue")
#BD_DENUE.cache()

BD_CENSO = spark.read.parquet(f"/content/gdrive/Shareddrives/Reto INEGI/Data_INEGI/SCINCE_Parquets/*.parquet")
BD_CENSO.createOrReplaceTempView("censo")
BD_CENSO.printSchema()

root
 |-- CVEGEO: string (nullable = true)
 |-- POB1: double (nullable = true)
 |-- POB2: double (nullable = true)
 |-- POB2_R: double (nullable = true)
 |-- POB4: double (nullable = true)
 |-- POB4_R: double (nullable = true)
 |-- POB5: double (nullable = true)
 |-- POB5_R: double (nullable = true)
 |-- POB6: double (nullable = true)
 |-- POB6_R: double (nullable = true)
 |-- POB7: double (nullable = true)
 |-- POB7_R: double (nullable = true)
 |-- POB8: double (nullable = true)
 |-- POB8_R: double (nullable = true)
 |-- POB9: double (nullable = true)
 |-- POB9_R: double (nullable = true)
 |-- POB10: double (nullable = true)
 |-- POB10_R: double (nullable = true)
 |-- POB11: double (nullable = true)
 |-- POB11_R: double (nullable = true)
 |-- POB12: double (nullable = true)
 |-- POB12_R: double (nullable = true)
 |-- POB13: double (nullable = true)
 |-- POB13_R: double (nullable = true)
 |-- POB14: double (nullable = true)
 |-- POB14_R: double (nullable = true)
 |-- POB15: double (nul

## Datasets

### Población

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

poblacion = spark.sql("""select 
                     CVEGEO,
                     if((isnull(POB1) or POB1 < 0 ), 0, POB1) AS POB1, 
                     if((isnull(POB31) or POB31 < 0 ), 0, POB31) AS POB31,
                     if((isnull(POB57) or POB57 < 0 ), 0, POB57) AS POB57,
                     if((isnull(POB2) or POB2 < 0 ), 0, POB2) AS POB2,
                     if((isnull(POB4) or POB4 < 0 ), 0, POB4) AS POB4,  
                     if((isnull(POB5) or POB5 < 0 ), 0, POB5) AS POB5,                                
                     if((isnull(POB7) or POB7 < 0 ), 0, POB7) AS POB7,
                     if((isnull(POB11) or POB11 < 0 ), 0, POB11) AS POB11,
                     if((isnull(POB14) or POB14 < 0 ), 0, POB14) AS POB14,
                     if((isnull(POB15) or POB15 < 0 ), 0, POB15) AS POB15,
                     if((isnull(POB16) or POB16 < 0 ), 0, POB16) AS POB16,
                     if((isnull(POB24) or POB24 < 0 ), 0, POB24) AS POB24
                     from censo""")

# Agrupación por Estados
poblacion = poblacion.withColumn("ESTADO", lit(None))
poblacion = poblacion.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
poblacion.show(100, False)

+----------------+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+------+
|CVEGEO          |POB1 |POB31|POB57|POB2|POB4|POB5|POB7|POB11|POB14|POB15|POB16|POB24|ESTADO|
+----------------+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+------+
|1500100010127009|48.0 |5.0  |0.0  |0.0 |0.0 |0.0 |0.0 |11.0 |17.0 |6.0  |0.0  |5.0  |Mexico|
|1500100010127010|182.0|9.0  |10.0 |7.0 |7.0 |15.0|10.0|49.0 |43.0 |19.0 |8.0  |24.0 |Mexico|
|1500100010127011|110.0|10.0 |9.0  |7.0 |0.0 |9.0 |3.0 |30.0 |21.0 |16.0 |7.0  |15.0 |Mexico|
|1500100010127030|159.0|14.0 |11.0 |10.0|8.0 |12.0|9.0 |34.0 |46.0 |19.0 |10.0 |11.0 |Mexico|
|1500100010127035|14.0 |0.0  |0.0  |0.0 |0.0 |0.0 |0.0 |0.0  |4.0  |3.0  |0.0  |0.0  |Mexico|
|1500100010269012|13.0 |0.0  |3.0  |0.0 |0.0 |0.0 |0.0 |4.0  |0.0  |4.0  |0.0  |0.0  |Mexico|
|1500100010288007|30.0 |0.0  |3.0  |0.0 |0.0 |6.0 |0.0 |7.0  |5.0  |6.0  |0.0  |0.0  |Mexico|
|1500100010324004|33.0 |0.0  |0.0  |0.0 |0.0 |6.0 |0.0 |10.0

In [ ]:
poblacion = poblacion.groupBy("ESTADO").sum(
    "POB1", "POB31", "POB57", "POB2", "POB4", "POB5", "POB7", "POB11", "POB14", "POB15", "POB16", "POB24")
poblacion.show(100, False)

+-------------------+-----------+----------+----------+---------+---------+---------+---------+----------+----------+----------+----------+----------+
|ESTADO             |sum(POB1)  |sum(POB31)|sum(POB57)|sum(POB2)|sum(POB4)|sum(POB5)|sum(POB7)|sum(POB11)|sum(POB14)|sum(POB15)|sum(POB16)|sum(POB24)|
+-------------------+-----------+----------+----------+---------+---------+---------+---------+----------+----------+----------+----------+----------+
|Mexico             |1.4428917E7|1061146.0 |804753.0  |526766.0 |634979.0 |1346058.0|676663.0 |3565802.0 |4190981.0 |1564606.0 |538262.0  |1067958.0 |
|Jalisco            |7364595.0  |538462.0  |340612.0  |289001.0 |335147.0 |721596.0 |332376.0 |1845751.0 |2035303.0 |687816.0  |235444.0  |558375.0  |
|Veracruz           |5012679.0  |327900.0  |276092.0  |143041.0 |183499.0 |441563.0 |193020.0 |1141976.0 |1385290.0 |554038.0  |183788.0  |469347.0  |
|Puebla             |4951465.0  |339855.0  |226030.0  |207821.0 |230030.0 |492903.0 |229526.0 

In [ ]:
poblacion.columns

['ESTADO',
 'sum(POB1)',
 'sum(POB31)',
 'sum(POB57)',
 'sum(POB2)',
 'sum(POB4)',
 'sum(POB5)',
 'sum(POB7)',
 'sum(POB11)',
 'sum(POB14)',
 'sum(POB15)',
 'sum(POB16)',
 'sum(POB24)']

In [ ]:
poblacion = poblacion.\
withColumnRenamed('sum(POB1)', 'POB1').\
withColumnRenamed('sum(POB31)', 'POB31').\
withColumnRenamed('sum(POB57)', 'POB57').\
withColumnRenamed('sum(POB2)', 'POB2').\
withColumnRenamed('sum(POB4)', 'POB4').\
withColumnRenamed('sum(POB5)', 'POB5').\
withColumnRenamed('sum(POB7)', 'POB7').\
withColumnRenamed('sum(POB11)', 'POB11').\
withColumnRenamed('sum(POB14)', 'POB14').\
withColumnRenamed('sum(POB15)', 'POB15').\
withColumnRenamed('sum(POB16)', 'POB16').\
withColumnRenamed('sum(POB24)', 'POB24')
poblacion.show(100, False)

+-------------------+-----------+---------+--------+--------+--------+---------+--------+---------+---------+---------+--------+---------+
|ESTADO             |POB1       |POB31    |POB57   |POB2    |POB4    |POB5     |POB7    |POB11    |POB14    |POB15    |POB16   |POB24    |
+-------------------+-----------+---------+--------+--------+--------+---------+--------+---------+---------+---------+--------+---------+
|Mexico             |1.4428917E7|1061146.0|804753.0|526766.0|634979.0|1346058.0|676663.0|3565802.0|4190981.0|1564606.0|538262.0|1067958.0|
|Jalisco            |7364595.0  |538462.0 |340612.0|289001.0|335147.0|721596.0 |332376.0|1845751.0|2035303.0|687816.0 |235444.0|558375.0 |
|Veracruz           |5012679.0  |327900.0 |276092.0|143041.0|183499.0|441563.0 |193020.0|1141976.0|1385290.0|554038.0 |183788.0|469347.0 |
|Puebla             |4951465.0  |339855.0 |226030.0|207821.0|230030.0|492903.0 |229526.0|1278947.0|1334546.0|450120.0 |144923.0|354655.0 |
|Oaxaca             |244042

In [ ]:
poblacion.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/poblacion.csv")

### Fecundidad

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

fecundidad = spark.sql("""select 
                     CVEGEO,
                     if((isnull(FEC1_R) or FEC1_R < 0 ), 0, FEC1_R) AS FEC1_R, 
                     if((isnull(FEC2_R) or FEC2_R < 0 ), 0, FEC2_R) AS FEC2_R,
                     if((isnull(FEC3_R) or FEC3_R < 0 ), 0, FEC3_R) AS FEC3_R
                     from censo""")

# Agrupación por Estados
fecundidad = fecundidad.withColumn("ESTADO", lit(None))
fecundidad = fecundidad.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
fecundidad.show(100, False)

+----------------+------+------+------+------+
|CVEGEO          |FEC1_R|FEC2_R|FEC3_R|ESTADO|
+----------------+------+------+------+------+
|1500100010127009|1.6   |1.0   |0.0   |Mexico|
|1500100010127010|1.8   |1.1   |0.0   |Mexico|
|1500100010127011|1.9   |1.2   |0.0   |Mexico|
|1500100010127030|1.9   |1.3   |0.0   |Mexico|
|1500100010127035|1.7   |0.3   |0.0   |Mexico|
|1500100010269012|3.5   |0.5   |0.0   |Mexico|
|1500100010288007|2.4   |2.0   |100.0 |Mexico|
|1500100010324004|2.2   |2.3   |100.0 |Mexico|
|1500100010339008|2.3   |1.8   |0.0   |Mexico|
|1500100370305001|2.5   |1.6   |0.0   |Mexico|
|1500100370305002|2.8   |2.0   |0.0   |Mexico|
|1500100370305009|1.9   |1.8   |0.0   |Mexico|
|1500100370305011|2.5   |1.9   |0.0   |Mexico|
|1500100370305017|2.6   |2.0   |0.0   |Mexico|
|1500100370305018|3.9   |2.8   |0.0   |Mexico|
|1500100370305033|4.1   |1.0   |0.0   |Mexico|
|1500100370305035|3.0   |1.6   |0.0   |Mexico|
|1500100370305038|3.0   |1.8   |33.3  |Mexico|
|150010037030

In [ ]:
fecundidad = fecundidad.groupBy("ESTADO").avg(
    "FEC1_R", "FEC2_R", "FEC3_R")
fecundidad.show(100, False)

+-------------------+------------------+------------------+------------------+
|ESTADO             |avg(FEC1_R)       |avg(FEC2_R)       |avg(FEC3_R)       |
+-------------------+------------------+------------------+------------------+
|Mexico             |1.7209867815960294|1.2410675058339191|6.085866417348759 |
|Jalisco            |1.730578005413901 |1.2198621200023967|5.737535297668703 |
|Veracruz           |1.71296572760372  |1.190528102076185 |5.582681216234908 |
|Puebla             |1.7888076521357092|1.2635025562091498|6.934962344016165 |
|Oaxaca             |1.6848201630050093|1.1274170210253012|4.787321344200307 |
|CDMX               |1.5361823275992048|0.9779366131413144|3.9536589039858288|
|Nuevo Leon         |1.5296071173453138|1.0998194687967127|5.6151594259366195|
|Chihuahua          |1.5529993776434612|1.1398816252397512|5.555518016587674 |
|Guanajuato         |1.647707938433558 |1.169755098946087 |6.216216544089773 |
|Michoacan          |1.7859228852074773|1.2393226799

In [ ]:
fecundidad.columns

['ESTADO', 'avg(FEC1_R)', 'avg(FEC2_R)', 'avg(FEC3_R)']

In [ ]:
fecundidad = fecundidad.\
withColumnRenamed('avg(FEC1_R)', 'FEC1_R').\
withColumnRenamed('avg(FEC2_R)', 'FEC2_R').\
withColumnRenamed('avg(FEC3_R)', 'FEC3_R')
fecundidad.show(100, False)

+-------------------+------------------+------------------+------------------+
|ESTADO             |FEC1_R            |FEC2_R            |FEC3_R            |
+-------------------+------------------+------------------+------------------+
|Mexico             |1.7209867815960294|1.2410675058339191|6.085866417348759 |
|Jalisco            |1.730578005413901 |1.2198621200023967|5.737535297668703 |
|Veracruz           |1.71296572760372  |1.190528102076185 |5.582681216234908 |
|Puebla             |1.7888076521357092|1.2635025562091498|6.934962344016165 |
|Oaxaca             |1.6848201630050093|1.1274170210253012|4.787321344200307 |
|CDMX               |1.5361823275992048|0.9779366131413144|3.9536589039858288|
|Nuevo Leon         |1.5296071173453138|1.0998194687967127|5.6151594259366195|
|Chihuahua          |1.5529993776434612|1.1398816252397512|5.555518016587674 |
|Guanajuato         |1.647707938433558 |1.169755098946087 |6.216216544089773 |
|Michoacan          |1.7859228852074773|1.2393226799

In [ ]:
fecundidad.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/fecundidad.csv")

### Mortalidad

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

mortalidad = spark.sql("""select 
                     CVEGEO,
                     if((isnull(MOR1_R) or MOR1_R < 0 ), 0, MOR1_R) AS MOR1_R
                     from censo""")

# Agrupación por Estados
mortalidad = mortalidad.withColumn("ESTADO", lit(None))
mortalidad = mortalidad.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
mortalidad.show(100, False)

+----------------+------+------+
|CVEGEO          |MOR1_R|ESTADO|
+----------------+------+------+
|1500100010127009|0.0   |Mexico|
|1500100010127010|5.9   |Mexico|
|1500100010127011|9.7   |Mexico|
|1500100010127030|2.9   |Mexico|
|1500100010127035|0.0   |Mexico|
|1500100010269012|23.8  |Mexico|
|1500100010288007|4.5   |Mexico|
|1500100010324004|3.8   |Mexico|
|1500100010339008|5.6   |Mexico|
|1500100370305001|10.3  |Mexico|
|1500100370305002|1.1   |Mexico|
|1500100370305009|8.7   |Mexico|
|1500100370305011|4.9   |Mexico|
|1500100370305017|0.0   |Mexico|
|1500100370305018|4.7   |Mexico|
|1500100370305033|8.1   |Mexico|
|1500100370305035|7.8   |Mexico|
|1500100370305038|7.2   |Mexico|
|1500100370305040|0.0   |Mexico|
|1500100370305048|0.0   |Mexico|
|1500100370305054|2.9   |Mexico|
|1500100370305060|0.0   |Mexico|
|150010037031A001|11.7  |Mexico|
|150010037031A010|0.0   |Mexico|
|150010037031A012|17.9  |Mexico|
|150010037031A017|25.0  |Mexico|
|150010037031A027|11.5  |Mexico|
|150010037

In [ ]:
mortalidad = mortalidad.groupBy("ESTADO").avg(
    "MOR1_R")
mortalidad.show(100, False)

+-------------------+------------------+
|ESTADO             |avg(MOR1_R)       |
+-------------------+------------------+
|Mexico             |5.518058057601496 |
|Jalisco            |5.060350737112405 |
|Veracruz           |5.721670765892624 |
|Puebla             |5.644719916442105 |
|Oaxaca             |5.275801736357158 |
|CDMX               |5.602444792576744 |
|Nuevo Leon         |3.9201545554905706|
|Chihuahua          |4.93856324539899  |
|Guanajuato         |4.791276063386162 |
|Michoacan          |5.138457704148946 |
|Guerrero           |5.840117480105645 |
|Tamaulipas         |4.644837498261421 |
|Sonora             |4.48467441447825  |
|Chiapas            |5.233199854386579 |
|Baja California    |5.0700559208727825|
|Coahuila           |4.111034658079766 |
|Sinaloa            |4.804155516393673 |
|Yucatan            |4.368987176814062 |
|Hidalgo            |4.640034003400348 |
|Queretaro          |4.560584730111216 |
|Morelos            |5.189821639151005 |
|San Luis Potosi

In [ ]:
mortalidad.columns

['ESTADO', 'avg(MOR1_R)']

In [ ]:
mortalidad = mortalidad.\
withColumnRenamed('avg(MOR1_R)', 'MOR1_R')
mortalidad.show(100, False)

+-------------------+------------------+
|ESTADO             |MOR1_R            |
+-------------------+------------------+
|Mexico             |5.518058057601496 |
|Jalisco            |5.060350737112405 |
|Veracruz           |5.721670765892624 |
|Puebla             |5.644719916442105 |
|Oaxaca             |5.275801736357158 |
|CDMX               |5.602444792576744 |
|Nuevo Leon         |3.9201545554905706|
|Chihuahua          |4.93856324539899  |
|Guanajuato         |4.791276063386162 |
|Michoacan          |5.138457704148946 |
|Guerrero           |5.840117480105645 |
|Tamaulipas         |4.644837498261421 |
|Sonora             |4.48467441447825  |
|Chiapas            |5.233199854386579 |
|Baja California    |5.0700559208727825|
|Coahuila           |4.111034658079766 |
|Sinaloa            |4.804155516393673 |
|Yucatan            |4.368987176814062 |
|Hidalgo            |4.640034003400348 |
|Queretaro          |4.560584730111216 |
|Morelos            |5.189821639151005 |
|San Luis Potosi

In [ ]:
mortalidad.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/mortalidad.csv")

### Migración

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

migracion = spark.sql("""select 
                     CVEGEO,
                     if((isnull(MIG1) or MIG1 < 0 ), 0, MIG1) AS MIG1, 
                     if((isnull(MIG2) or MIG2 < 0 ), 0, MIG2) AS MIG2,
                     if((isnull(MIG3) or MIG3 < 0 ), 0, MIG3) AS MIG3,
                     if((isnull(MIG4) or MIG4 < 0 ), 0, MIG4) AS MIG4,
                     if((isnull(MIG5) or MIG5 < 0 ), 0, MIG5) AS MIG5,  
                     if((isnull(MIG6) or MIG6 < 0 ), 0, MIG6) AS MIG6,                                
                     if((isnull(MIG7) or MIG7 < 0 ), 0, MIG7) AS MIG7
                     from censo""")

# Agrupación por Estados
migracion = migracion.withColumn("ESTADO", lit(None))
migracion = migracion.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
migracion.show(100, False)

+----------------+-----+-----+-----+-----+----+----+----+------+
|CVEGEO          |MIG1 |MIG2 |MIG3 |MIG4 |MIG5|MIG6|MIG7|ESTADO|
+----------------+-----+-----+-----+-----+----+----+----+------+
|1500100010127009|45.0 |23.0 |22.0 |0.0  |0.0 |0.0 |0.0 |Mexico|
|1500100010127010|141.0|81.0 |60.0 |37.0 |20.0|17.0|4.0 |Mexico|
|1500100010127011|102.0|58.0 |44.0 |8.0  |3.0 |5.0 |0.0 |Mexico|
|1500100010127030|136.0|78.0 |58.0 |22.0 |11.0|11.0|0.0 |Mexico|
|1500100010127035|8.0  |5.0  |3.0  |6.0  |3.0 |3.0 |0.0 |Mexico|
|1500100010269012|13.0 |7.0  |6.0  |0.0  |0.0 |0.0 |0.0 |Mexico|
|1500100010288007|23.0 |10.0 |13.0 |7.0  |4.0 |3.0 |0.0 |Mexico|
|1500100010324004|28.0 |13.0 |15.0 |4.0  |0.0 |3.0 |0.0 |Mexico|
|1500100010339008|35.0 |21.0 |14.0 |0.0  |0.0 |0.0 |0.0 |Mexico|
|1500100370305001|105.0|56.0 |49.0 |0.0  |0.0 |0.0 |0.0 |Mexico|
|1500100370305002|94.0 |44.0 |50.0 |0.0  |0.0 |0.0 |0.0 |Mexico|
|1500100370305009|64.0 |39.0 |25.0 |5.0  |0.0 |3.0 |0.0 |Mexico|
|1500100370305011|137.0|6

In [ ]:
migracion = migracion.groupBy("ESTADO").sum(
    "MIG1", "MIG2", "MIG3", "MIG4", "MIG5", "MIG6", "MIG7")
migracion.show(100, False)

+-------------------+---------+---------+---------+---------+---------+---------+---------+
|ESTADO             |sum(MIG1)|sum(MIG2)|sum(MIG3)|sum(MIG4)|sum(MIG5)|sum(MIG6)|sum(MIG7)|
+-------------------+---------+---------+---------+---------+---------+---------+---------+
|Mexico             |9366228.0|4765731.0|4594355.0|4907218.0|2582575.0|2296988.0|31952.0  |
|Jalisco            |6241674.0|3194392.0|3044542.0|934940.0 |455141.0 |437626.0 |49504.0  |
|Veracruz           |4416650.0|2330038.0|2082854.0|511608.0 |250313.0 |213279.0 |5320.0   |
|Puebla             |4336536.0|2249400.0|2085322.0|524769.0 |274358.0 |219284.0 |13055.0  |
|Oaxaca             |2171321.0|1144142.0|1024334.0|172450.0 |78955.0  |68402.0  |5167.0   |
|CDMX               |7341014.0|3777179.0|3562943.0|1649147.0|913121.0 |717443.0 |87358.0  |
|Nuevo Leon         |4186764.0|2097343.0|2087024.0|1272361.0|629966.0 |624536.0 |30330.0  |
|Chihuahua          |2630660.0|1340518.0|1286728.0|518879.0 |244398.0 |245619.0 

In [ ]:
migracion.columns

['ESTADO',
 'sum(MIG1)',
 'sum(MIG2)',
 'sum(MIG3)',
 'sum(MIG4)',
 'sum(MIG5)',
 'sum(MIG6)',
 'sum(MIG7)']

In [ ]:
migracion = migracion.\
withColumnRenamed('sum(MIG1)', 'MIG1').\
withColumnRenamed('sum(MIG2)', 'MIG2').\
withColumnRenamed('sum(MIG3)', 'MIG3').\
withColumnRenamed('sum(MIG4)', 'MIG4').\
withColumnRenamed('sum(MIG5)', 'MIG5').\
withColumnRenamed('sum(MIG6)', 'MIG6').\
withColumnRenamed('sum(MIG7)', 'MIG7')
migracion.show(100, False)

+-------------------+---------+---------+---------+---------+---------+---------+--------+
|ESTADO             |MIG1     |MIG2     |MIG3     |MIG4     |MIG5     |MIG6     |MIG7    |
+-------------------+---------+---------+---------+---------+---------+---------+--------+
|Mexico             |9366228.0|4765731.0|4594355.0|4907218.0|2582575.0|2296988.0|31952.0 |
|Jalisco            |6241674.0|3194392.0|3044542.0|934940.0 |455141.0 |437626.0 |49504.0 |
|Veracruz           |4416650.0|2330038.0|2082854.0|511608.0 |250313.0 |213279.0 |5320.0  |
|Puebla             |4336536.0|2249400.0|2085322.0|524769.0 |274358.0 |219284.0 |13055.0 |
|Oaxaca             |2171321.0|1144142.0|1024334.0|172450.0 |78955.0  |68402.0  |5167.0  |
|CDMX               |7341014.0|3777179.0|3562943.0|1649147.0|913121.0 |717443.0 |87358.0 |
|Nuevo Leon         |4186764.0|2097343.0|2087024.0|1272361.0|629966.0 |624536.0 |30330.0 |
|Chihuahua          |2630660.0|1340518.0|1286728.0|518879.0 |244398.0 |245619.0 |72708.0 |

In [ ]:
migracion.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/migracion.csv")

### Hablantes de Lengua Indígena

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

hablantes = spark.sql("""select 
                     CVEGEO,
                     if((isnull(INDI10) or INDI10 < 0 ), 0, INDI10) AS INDI10, 
                     if((isnull(INDI13) or INDI13 < 0 ), 0, INDI13) AS INDI13,
                     if((isnull(INDI16) or INDI16 < 0 ), 0, INDI16) AS INDI16,
                     if((isnull(INDI20) or INDI20 < 0 ), 0, INDI20) AS INDI20
                     from censo""")

# Agrupación por Estados
hablantes = hablantes.withColumn("ESTADO", lit(None))
hablantes = hablantes.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
hablantes.show(100, False)

+----------------+------+------+------+------+------+
|CVEGEO          |INDI10|INDI13|INDI16|INDI20|ESTADO|
+----------------+------+------+------+------+------+
|1500100010127009|5.0   |0.0   |5.0   |5.0   |Mexico|
|1500100010127010|8.0   |0.0   |8.0   |26.0  |Mexico|
|1500100010127011|0.0   |0.0   |0.0   |0.0   |Mexico|
|1500100010127030|8.0   |0.0   |8.0   |17.0  |Mexico|
|1500100010127035|0.0   |0.0   |0.0   |0.0   |Mexico|
|1500100010269012|0.0   |0.0   |0.0   |0.0   |Mexico|
|1500100010288007|0.0   |0.0   |0.0   |0.0   |Mexico|
|1500100010324004|0.0   |0.0   |0.0   |0.0   |Mexico|
|1500100010339008|0.0   |0.0   |0.0   |5.0   |Mexico|
|1500100370305001|36.0  |0.0   |36.0  |83.0  |Mexico|
|1500100370305002|42.0  |0.0   |42.0  |85.0  |Mexico|
|1500100370305009|27.0  |0.0   |27.0  |57.0  |Mexico|
|1500100370305011|56.0  |0.0   |56.0  |123.0 |Mexico|
|1500100370305017|8.0   |0.0   |8.0   |16.0  |Mexico|
|1500100370305018|14.0  |0.0   |14.0  |24.0  |Mexico|
|1500100370305033|13.0  |0.0

In [ ]:
hablantes = hablantes.groupBy("ESTADO").sum(
    "INDI10", "INDI13", "INDI16", "INDI20")
hablantes.show(100, False)

+-------------------+-----------+-----------+-----------+-----------+
|ESTADO             |sum(INDI10)|sum(INDI13)|sum(INDI16)|sum(INDI20)|
+-------------------+-----------+-----------+-----------+-----------+
|Mexico             |244854.0   |962.0      |236578.0   |709648.0   |
|Jalisco            |28048.0    |211.0      |27100.0    |73366.0    |
|Veracruz           |161683.0   |6204.0     |153076.0   |328699.0   |
|Puebla             |247055.0   |6673.0     |236199.0   |529013.0   |
|Oaxaca             |501418.0   |31150.0    |461359.0   |836441.0   |
|CDMX               |97717.0    |120.0      |95763.0    |273095.0   |
|Nuevo Leon         |61965.0    |50.0       |60605.0    |152676.0   |
|Chihuahua          |19909.0    |164.0      |19048.0    |53335.0    |
|Guanajuato         |5615.0     |48.0       |5160.0     |19915.0    |
|Michoacan          |92641.0    |4843.0     |86289.0    |154624.0   |
|Guerrero           |130879.0   |9569.0     |118626.0   |231451.0   |
|Tamaulipas         

In [ ]:
hablantes.columns

['ESTADO', 'sum(INDI10)', 'sum(INDI13)', 'sum(INDI16)', 'sum(INDI20)']

In [ ]:
hablantes = hablantes.\
withColumnRenamed('sum(INDI10)', 'INDI10').\
withColumnRenamed('sum(INDI13)', 'INDI13').\
withColumnRenamed('sum(INDI16)', 'INDI16').\
withColumnRenamed('sum(INDI20)', 'INDI20')
hablantes.show(100, False)

+-------------------+--------+-------+--------+--------+
|ESTADO             |INDI10  |INDI13 |INDI16  |INDI20  |
+-------------------+--------+-------+--------+--------+
|Mexico             |244854.0|962.0  |236578.0|709648.0|
|Jalisco            |28048.0 |211.0  |27100.0 |73366.0 |
|Veracruz           |161683.0|6204.0 |153076.0|328699.0|
|Puebla             |247055.0|6673.0 |236199.0|529013.0|
|Oaxaca             |501418.0|31150.0|461359.0|836441.0|
|CDMX               |97717.0 |120.0  |95763.0 |273095.0|
|Nuevo Leon         |61965.0 |50.0   |60605.0 |152676.0|
|Chihuahua          |19909.0 |164.0  |19048.0 |53335.0 |
|Guanajuato         |5615.0  |48.0   |5160.0  |19915.0 |
|Michoacan          |92641.0 |4843.0 |86289.0 |154624.0|
|Guerrero           |130879.0|9569.0 |118626.0|231451.0|
|Tamaulipas         |11250.0 |9.0    |11039.0 |52699.0 |
|Sonora             |20413.0 |102.0  |19812.0 |63563.0 |
|Chiapas            |275657.0|35393.0|234598.0|438782.0|
|Baja California    |26828.0 |2

In [ ]:
hablantes.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/hablantes.csv")

### Discapacidad

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

discapacidad = spark.sql("""select 
                     CVEGEO,
                     if((isnull(DISC1) or DISC1 < 0 ), 0, DISC1) AS DISC1, 
                     if((isnull(DISC2) or DISC2 < 0 ), 0, DISC2) AS DISC2,
                     if((isnull(DISC3) or DISC3 < 0 ), 0, DISC3) AS DISC3,
                     if((isnull(DISC4) or DISC4 < 0 ), 0, DISC4) AS DISC4,
                     if((isnull(DISC5) or DISC5 < 0 ), 0, DISC5) AS DISC5,  
                     if((isnull(DISC6) or DISC6 < 0 ), 0, DISC6) AS DISC6,                                
                     if((isnull(DISC7) or DISC7 < 0 ), 0, DISC7) AS DISC7,
                     if((isnull(DISC8) or DISC8 < 0 ), 0, DISC8) AS DISC8,
                     if((isnull(DISC9) or DISC9 < 0 ), 0, DISC9) AS DISC9,
                     if((isnull(DISC10) or DISC10 < 0 ), 0, DISC10) AS DISC10,
                     if((isnull(DISC11) or DISC11 < 0 ), 0, DISC11) AS DISC11,
                     if((isnull(DISC12) or DISC12 < 0 ), 0, DISC12) AS DISC12,
                     if((isnull(DISC13) or DISC13 < 0 ), 0, DISC13) AS DISC13,
                     if((isnull(DISC14) or DISC14 < 0 ), 0, DISC14) AS DISC14
                     from censo""")

# Agrupación por Estados
discapacidad = discapacidad.withColumn("ESTADO", lit(None))
discapacidad = discapacidad.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
discapacidad.show(100, False)

+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|CVEGEO          |DISC1|DISC2|DISC3|DISC4|DISC5|DISC6|DISC7|DISC8|DISC9|DISC10|DISC11|DISC12|DISC13|DISC14|ESTADO|
+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|1500100010127009|0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0   |0.0   |0.0   |0.0   |40.0  |Mexico|
|1500100010127010|15.0 |8.0  |7.0  |3.0  |6.0  |6.0  |4.0  |6.0  |0.0  |4.0   |0.0   |4.0   |0.0   |121.0 |Mexico|
|1500100010127011|5.0  |3.0  |0.0  |0.0  |3.0  |0.0  |4.0  |0.0  |0.0  |0.0   |0.0   |0.0   |0.0   |98.0  |Mexico|
|1500100010127030|14.0 |9.0  |5.0  |0.0  |7.0  |6.0  |6.0  |8.0  |0.0  |0.0   |0.0   |4.0   |3.0   |106.0 |Mexico|
|1500100010127035|0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0   |0.0   |0.0   |0.0   |13.0  |Mexico|
|1500100010269012|4.0  |3.0  |0.0  |0.0  |0.0  |0.0  |4.0  |0.0  |0.0  |0.0   |0

In [ ]:
discapacidad = discapacidad.groupBy("ESTADO").sum(
    "DISC1", "DISC2", "DISC3", "DISC4", "DISC5", "DISC6", "DISC7", "DISC8", "DISC9", "DISC10", "DISC11", "DISC12", "DISC13", "DISC14")
discapacidad.show(100, False)

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+
|ESTADO             |sum(DISC1)|sum(DISC2)|sum(DISC3)|sum(DISC4)|sum(DISC5)|sum(DISC6)|sum(DISC7)|sum(DISC8)|sum(DISC9)|sum(DISC10)|sum(DISC11)|sum(DISC12)|sum(DISC13)|sum(DISC14)|
+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+
|Mexico             |597838.0  |288061.0  |236882.0  |26653.0   |226802.0  |236857.0  |222130.0  |249921.0  |40054.0   |83700.0    |54053.0    |56925.0    |117052.0   |1.1969442E7|
|Jalisco            |291433.0  |135001.0  |105910.0  |9357.0    |86146.0   |128696.0  |126801.0  |92933.0   |17266.0   |30013.0    |28901.0    |28398.0    |55528.0    |6190064.0  |
|Veracruz           |238837.0  |109337.0  |79889.0   |6253.0    |63554.0   |106711.0  |86812.0 

In [ ]:
discapacidad.columns

['ESTADO',
 'sum(DISC1)',
 'sum(DISC2)',
 'sum(DISC3)',
 'sum(DISC4)',
 'sum(DISC5)',
 'sum(DISC6)',
 'sum(DISC7)',
 'sum(DISC8)',
 'sum(DISC9)',
 'sum(DISC10)',
 'sum(DISC11)',
 'sum(DISC12)',
 'sum(DISC13)',
 'sum(DISC14)']

In [ ]:
discapacidad = discapacidad.\
withColumnRenamed('sum(DISC1)', 'DISC1').\
withColumnRenamed('sum(DISC2)', 'DISC2').\
withColumnRenamed('sum(DISC3)', 'DISC3').\
withColumnRenamed('sum(DISC4)', 'DISC4').\
withColumnRenamed('sum(DISC5)', 'DISC5').\
withColumnRenamed('sum(DISC6)', 'DISC6').\
withColumnRenamed('sum(DISC7)', 'DISC7').\
withColumnRenamed('sum(DISC8)', 'DISC8').\
withColumnRenamed('sum(DISC9)', 'DISC9').\
withColumnRenamed('sum(DISC10)', 'DISC10').\
withColumnRenamed('sum(DISC11)', 'DISC11').\
withColumnRenamed('sum(DISC12)', 'DISC12').\
withColumnRenamed('sum(DISC13)', 'DISC13').\
withColumnRenamed('sum(DISC14)', 'DISC14')
discapacidad.show(100, False)

+-------------------+--------+--------+--------+-------+--------+--------+--------+--------+-------+-------+-------+-------+--------+-----------+
|ESTADO             |DISC1   |DISC2   |DISC3   |DISC4  |DISC5   |DISC6   |DISC7   |DISC8   |DISC9  |DISC10 |DISC11 |DISC12 |DISC13  |DISC14     |
+-------------------+--------+--------+--------+-------+--------+--------+--------+--------+-------+-------+-------+-------+--------+-----------+
|Mexico             |597838.0|288061.0|236882.0|26653.0|226802.0|236857.0|222130.0|249921.0|40054.0|83700.0|54053.0|56925.0|117052.0|1.1969442E7|
|Jalisco            |291433.0|135001.0|105910.0|9357.0 |86146.0 |128696.0|126801.0|92933.0 |17266.0|30013.0|28901.0|28398.0|55528.0 |6190064.0  |
|Veracruz           |238837.0|109337.0|79889.0 |6253.0 |63554.0 |106711.0|86812.0 |86568.0 |13315.0|28389.0|20344.0|21234.0|29888.0 |4034834.0  |
|Puebla             |181354.0|80853.0 |64009.0 |6329.0 |54540.0 |72579.0 |65279.0 |61229.0 |11274.0|23018.0|16550.0|16369.0|

In [ ]:
discapacidad.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/discapacidad.csv")

### Educación

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

educacion = spark.sql("""select 
                     CVEGEO,
                     if((isnull(EDU1) or EDU1 < 0 ), 0, EDU1) AS EDU1,
                     if((isnull(EDU4) or EDU4 < 0 ), 0, EDU4) AS EDU4,
                     if((isnull(EDU7) or EDU7 < 0 ), 0, EDU7) AS EDU7,
                     if((isnull(EDU10) or EDU10 < 0 ), 0, EDU10) AS EDU10,
                     if((isnull(EDU13) or EDU13 < 0 ), 0, EDU13) AS EDU13,
                     if((isnull(EDU16) or EDU16 < 0 ), 0, EDU16) AS EDU16,
                     if((isnull(EDU19) or EDU19 < 0 ), 0, EDU19) AS EDU19,
                     if((isnull(EDU22) or EDU22 < 0 ), 0, EDU22) AS EDU22,
                     if((isnull(EDU25) or EDU25 < 0 ), 0, EDU25) AS EDU25,
                     if((isnull(EDU28) or EDU28 < 0 ), 0, EDU28) AS EDU28,
                     if((isnull(EDU31) or EDU31 < 0 ), 0, EDU31) AS EDU31
                     from censo""")

# Agrupación por Estados
educacion = educacion.withColumn("ESTADO", lit(None))
educacion = educacion.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
educacion.show(100, False)

+----------------+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+------+
|CVEGEO          |EDU1|EDU4|EDU7|EDU10|EDU13|EDU16|EDU19|EDU22|EDU25|EDU28|EDU31|ESTADO|
+----------------+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+------+
|1500100010127009|0.0 |0.0 |0.0 |0.0  |0.0  |0.0  |0.0  |0.0  |41.0 |0.0  |0.0  |Mexico|
|1500100010127010|5.0 |0.0 |15.0|0.0  |10.0 |0.0  |20.0 |0.0  |137.0|5.0  |4.0  |Mexico|
|1500100010127011|0.0 |0.0 |9.0 |0.0  |0.0  |0.0  |7.0  |0.0  |87.0 |0.0  |0.0  |Mexico|
|1500100010127030|3.0 |5.0 |12.0|0.0  |9.0  |0.0  |15.0 |0.0  |119.0|0.0  |4.0  |Mexico|
|1500100010127035|0.0 |0.0 |0.0 |0.0  |0.0  |0.0  |0.0  |0.0  |11.0 |0.0  |0.0  |Mexico|
|1500100010269012|0.0 |0.0 |0.0 |0.0  |0.0  |0.0  |0.0  |0.0  |11.0 |0.0  |0.0  |Mexico|
|1500100010288007|0.0 |0.0 |6.0 |0.0  |0.0  |0.0  |5.0  |0.0  |18.0 |0.0  |0.0  |Mexico|
|1500100010324004|0.0 |0.0 |6.0 |0.0  |0.0  |0.0  |7.0  |0.0  |20.0 |0.0  |0.0  |Mexico|
|1500100010339008|4.0

In [ ]:
educacion = educacion.groupBy("ESTADO").sum(
    'EDU1', 'EDU4', 'EDU7', 'EDU10', 'EDU13',
    'EDU16', 'EDU19', 'EDU22', 'EDU25', 'EDU28',
    'EDU31')
educacion.show(100, False)

+-------------------+---------+---------+---------+----------+----------+----------+----------+----------+-----------+----------+----------+
|ESTADO             |sum(EDU1)|sum(EDU4)|sum(EDU7)|sum(EDU10)|sum(EDU13)|sum(EDU16)|sum(EDU19)|sum(EDU22)|sum(EDU25) |sum(EDU28)|sum(EDU31)|
+-------------------+---------+---------+---------+----------+----------+----------+----------+----------+-----------+----------+----------+
|Mexico             |359877.0 |195199.0 |1277982.0|29211.0   |621327.0  |17436.0   |1591985.0 |4411.0    |1.0758829E7|191996.0  |274710.0  |
|Jalisco            |177436.0 |100126.0 |682128.0 |15003.0   |292105.0  |12661.0   |826019.0  |4405.0    |5325815.0  |90691.0   |133755.0  |
|Veracruz           |97059.0  |42649.0  |420424.0 |5268.0    |172737.0  |4534.0    |514449.0  |2278.0    |3643167.0  |159023.0  |185983.0  |
|Puebla             |147331.0 |47501.0  |471777.0 |7009.0    |202738.0  |9277.0    |569674.0  |3079.0    |3463111.0  |147485.0  |150590.0  |
|Oaxaca      

In [ ]:
educacion.columns

['ESTADO',
 'sum(EDU1)',
 'sum(EDU4)',
 'sum(EDU7)',
 'sum(EDU10)',
 'sum(EDU13)',
 'sum(EDU16)',
 'sum(EDU19)',
 'sum(EDU22)',
 'sum(EDU25)',
 'sum(EDU28)',
 'sum(EDU31)']

In [ ]:
educacion = educacion.\
withColumnRenamed('sum(EDU1)', 'EDU1').\
withColumnRenamed('sum(EDU4)', 'EDU4').\
withColumnRenamed('sum(EDU7)', 'EDU7').\
withColumnRenamed('sum(EDU10)', 'EDU10').\
withColumnRenamed('sum(EDU13)', 'EDU13').\
withColumnRenamed('sum(EDU16)', 'EDU16').\
withColumnRenamed('sum(EDU19)', 'EDU19').\
withColumnRenamed('sum(EDU22)', 'EDU22').\
withColumnRenamed('sum(EDU25)', 'EDU25').\
withColumnRenamed('sum(EDU28)', 'EDU28').\
withColumnRenamed('sum(EDU31)', 'EDU31')
educacion.show(100, False)

+-------------------+--------+--------+---------+-------+--------+-------+---------+------+-----------+--------+--------+
|ESTADO             |EDU1    |EDU4    |EDU7     |EDU10  |EDU13   |EDU16  |EDU19    |EDU22 |EDU25      |EDU28   |EDU31   |
+-------------------+--------+--------+---------+-------+--------+-------+---------+------+-----------+--------+--------+
|Mexico             |359877.0|195199.0|1277982.0|29211.0|621327.0|17436.0|1591985.0|4411.0|1.0758829E7|191996.0|274710.0|
|Jalisco            |177436.0|100126.0|682128.0 |15003.0|292105.0|12661.0|826019.0 |4405.0|5325815.0  |90691.0 |133755.0|
|Veracruz           |97059.0 |42649.0 |420424.0 |5268.0 |172737.0|4534.0 |514449.0 |2278.0|3643167.0  |159023.0|185983.0|
|Puebla             |147331.0|47501.0 |471777.0 |7009.0 |202738.0|9277.0 |569674.0 |3079.0|3463111.0  |147485.0|150590.0|
|Oaxaca             |65456.0 |13173.0 |220654.0 |1650.0 |87274.0 |2333.0 |269873.0 |1225.0|1614880.0  |122062.0|114453.0|
|CDMX               |200

In [ ]:
educacion.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/educacion.csv")

### Economía

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

economia = spark.sql("""select 
                     CVEGEO,
                     if((isnull(ECO1) or ECO1 < 0 ), 0, ECO1) AS ECO1,
                     if((isnull(ECO2) or ECO2 < 0 ), 0, ECO2) AS ECO2,
                     if((isnull(ECO3) or ECO3 < 0 ), 0, ECO3) AS ECO3,

                     if((isnull(ECO28) or ECO28 < 0 ), 0, ECO28) AS ECO28,
                     if((isnull(ECO29) or ECO29 < 0 ), 0, ECO29) AS ECO29,
                     if((isnull(ECO30) or ECO30 < 0 ), 0, ECO30) AS ECO30,

                     if((isnull(ECO34) or ECO34 < 0 ), 0, ECO34) AS ECO34,
                     if((isnull(ECO37) or ECO37 < 0 ), 0, ECO37) AS ECO37,
                     if((isnull(ECO40) or ECO40 < 0 ), 0, ECO40) AS ECO40,
                     if((isnull(ECO43) or ECO43 < 0 ), 0, ECO43) AS ECO43
                     from censo""")

# Agrupación por Estados
economia = economia.withColumn("ESTADO", lit(None))
economia = economia.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
economia.show(100, False)

+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
|CVEGEO          |ECO1 |ECO2 |ECO3 |ECO28|ECO29|ECO30|ECO34|ECO37|ECO40|ECO43|ESTADO|
+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
|1500100010127009|31.0 |12.0 |19.0 |12.0 |10.0 |0.0  |7.0  |4.0  |0.0  |0.0  |Mexico|
|1500100010127010|93.0 |55.0 |38.0 |59.0 |38.0 |21.0 |26.0 |17.0 |0.0  |11.0 |Mexico|
|1500100010127011|64.0 |31.0 |33.0 |28.0 |19.0 |9.0  |12.0 |8.0  |0.0  |4.0  |Mexico|
|1500100010127030|86.0 |46.0 |40.0 |43.0 |28.0 |15.0 |15.0 |16.0 |4.0  |0.0  |Mexico|
|1500100010127035|9.0  |5.0  |4.0  |3.0  |0.0  |0.0  |0.0  |0.0  |0.0  |0.0  |Mexico|
|1500100010269012|5.0  |0.0  |3.0  |7.0  |4.0  |3.0  |0.0  |0.0  |0.0  |4.0  |Mexico|
|1500100010288007|13.0 |5.0  |8.0  |7.0  |4.0  |3.0  |0.0  |5.0  |0.0  |0.0  |Mexico|
|1500100010324004|13.0 |5.0  |8.0  |11.0 |7.0  |4.0  |5.0  |4.0  |0.0  |0.0  |Mexico|
|1500100010339008|17.0 |10.0 |7.0  |6.0  |6.0  |0.0  |

In [ ]:
economia = economia.groupBy("ESTADO").sum(
    'ECO1', 'ECO2', 'ECO3', 'ECO28', 'ECO29', 'ECO30', 
    'ECO34', 'ECO37', 'ECO40', 'ECO43')
economia.show(100, False)

+-------------------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+
|ESTADO             |sum(ECO1)|sum(ECO2)|sum(ECO3)|sum(ECO28)|sum(ECO29)|sum(ECO30)|sum(ECO34)|sum(ECO37)|sum(ECO40)|sum(ECO43)|
+-------------------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+
|Mexico             |7304051.0|3066532.0|4228973.0|4417876.0 |3022826.0 |1367707.0 |1699881.0 |1837932.0 |62221.0   |260117.0  |
|Jalisco            |3791957.0|1604008.0|2177548.0|2020355.0 |1382759.0 |608502.0  |683517.0  |812893.0  |37381.0   |156215.0  |
|Veracruz           |2449672.0|1045516.0|1387998.0|1618833.0 |1115582.0 |470434.0  |521473.0  |684241.0  |35734.0   |92478.0   |
|Puebla             |2428813.0|1019589.0|1398809.0|1459516.0 |1026953.0 |408904.0  |542509.0  |632168.0  |29430.0   |76905.0   |
|Oaxaca             |1127446.0|480673.0 |630289.0 |748955.0  |508205.0  |214610.0  |232223.0  |31

In [ ]:
economia.columns

['ESTADO',
 'sum(ECO1)',
 'sum(ECO2)',
 'sum(ECO3)',
 'sum(ECO28)',
 'sum(ECO29)',
 'sum(ECO30)',
 'sum(ECO34)',
 'sum(ECO37)',
 'sum(ECO40)',
 'sum(ECO43)']

In [ ]:
economia = economia.\
withColumnRenamed('sum(ECO1)', 'ECO1').\
withColumnRenamed('sum(ECO2)', 'ECO2').\
withColumnRenamed('sum(ECO3)', 'ECO3').\
withColumnRenamed('sum(ECO28)', 'ECO28').\
withColumnRenamed('sum(ECO29)', 'ECO29').\
withColumnRenamed('sum(ECO30)', 'ECO30').\
withColumnRenamed('sum(ECO34)', 'ECO34').\
withColumnRenamed('sum(ECO37)', 'ECO37').\
withColumnRenamed('sum(ECO40)', 'ECO40').\
withColumnRenamed('sum(ECO43)', 'ECO43')
economia.show(100, False)

+-------------------+---------+---------+---------+---------+---------+---------+---------+---------+-------+--------+
|ESTADO             |ECO1     |ECO2     |ECO3     |ECO28    |ECO29    |ECO30    |ECO34    |ECO37    |ECO40  |ECO43   |
+-------------------+---------+---------+---------+---------+---------+---------+---------+---------+-------+--------+
|Mexico             |7304051.0|3066532.0|4228973.0|4417876.0|3022826.0|1367707.0|1699881.0|1837932.0|62221.0|260117.0|
|Jalisco            |3791957.0|1604008.0|2177548.0|2020355.0|1382759.0|608502.0 |683517.0 |812893.0 |37381.0|156215.0|
|Veracruz           |2449672.0|1045516.0|1387998.0|1618833.0|1115582.0|470434.0 |521473.0 |684241.0 |35734.0|92478.0 |
|Puebla             |2428813.0|1019589.0|1398809.0|1459516.0|1026953.0|408904.0 |542509.0 |632168.0 |29430.0|76905.0 |
|Oaxaca             |1127446.0|480673.0 |630289.0 |748955.0 |508205.0 |214610.0 |232223.0 |313511.0 |24214.0|62401.0 |
|CDMX               |5053214.0|2296503.0|2754494

In [ ]:
economia.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/economia.csv")

### Salud

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

salud = spark.sql("""select 
                     CVEGEO,
                     if((isnull(SALUD1) or SALUD1 < 0 ), 0, SALUD1) AS SALUD1,
                     if((isnull(SALUD2) or SALUD2 < 0 ), 0, SALUD2) AS SALUD2,
                     if((isnull(SALUD3) or SALUD3 < 0 ), 0, SALUD3) AS SALUD3,
                     if((isnull(SALUD4) or SALUD4 < 0 ), 0, SALUD4) AS SALUD4,
                     if((isnull(SALUD5) or SALUD5 < 0 ), 0, SALUD5) AS SALUD5,
                     if((isnull(SALUD6) or SALUD6 < 0 ), 0, SALUD6) AS SALUD6
                     from censo""")

# Agrupación por Estados
salud = salud.withColumn("ESTADO", lit(None))
salud = salud.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
salud.show(100, False)

+----------------+------+------+------+------+------+------+------+
|CVEGEO          |SALUD1|SALUD2|SALUD3|SALUD4|SALUD5|SALUD6|ESTADO|
+----------------+------+------+------+------+------+------+------+
|1500100010127009|33.0  |15.0  |4.0   |13.0  |14.0  |0.0   |Mexico|
|1500100010127010|140.0 |42.0  |20.0  |5.0   |67.0  |0.0   |Mexico|
|1500100010127011|66.0  |44.0  |7.0   |24.0  |25.0  |0.0   |Mexico|
|1500100010127030|99.0  |60.0  |23.0  |17.0  |40.0  |0.0   |Mexico|
|1500100010127035|10.0  |4.0   |0.0   |0.0   |0.0   |0.0   |Mexico|
|1500100010269012|11.0  |0.0   |0.0   |0.0   |7.0   |0.0   |Mexico|
|1500100010288007|17.0  |13.0  |6.0   |4.0   |7.0   |0.0   |Mexico|
|1500100010324004|20.0  |12.0  |0.0   |5.0   |7.0   |0.0   |Mexico|
|1500100010339008|31.0  |5.0   |0.0   |9.0   |22.0  |0.0   |Mexico|
|1500100370305001|78.0  |28.0  |0.0   |5.0   |71.0  |0.0   |Mexico|
|1500100370305002|18.0  |76.0  |5.0   |6.0   |6.0   |0.0   |Mexico|
|1500100370305009|28.0  |41.0  |0.0   |0.0   |26

In [ ]:
salud = salud.groupBy("ESTADO").sum(
    'SALUD1', 'SALUD2', 'SALUD3', 'SALUD4', 'SALUD5', 'SALUD6')
salud.show(100, False)

+-------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|ESTADO             |sum(SALUD1)|sum(SALUD2)|sum(SALUD3)|sum(SALUD4)|sum(SALUD5)|sum(SALUD6)|
+-------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|Mexico             |9542506.0  |4806894.0  |5582779.0  |744254.0   |2285966.0  |108125.0   |
|Jalisco            |5128892.0  |2136824.0  |3865818.0  |178093.0   |808097.0   |15637.0    |
|Veracruz           |3547623.0  |1413375.0  |1797471.0  |273741.0   |1173513.0  |191960.0   |
|Puebla             |3377488.0  |1519386.0  |1311769.0  |180448.0   |1625086.0  |24270.0    |
|Oaxaca             |1596748.0  |769859.0   |428215.0   |209228.0   |840017.0   |41407.0    |
|CDMX               |6626503.0  |2472027.0  |3864930.0  |1111978.0  |1165768.0  |86436.0    |
|Nuevo Leon         |4485750.0  |1019247.0  |3513884.0  |153082.0   |443558.0   |24454.0    |
|Chihuahua          |2771221.0  |478047.0   |1936334.0  |145

In [ ]:
salud.columns

['ESTADO',
 'sum(SALUD1)',
 'sum(SALUD2)',
 'sum(SALUD3)',
 'sum(SALUD4)',
 'sum(SALUD5)',
 'sum(SALUD6)']

In [ ]:
salud = salud.\
withColumnRenamed('sum(SALUD1)', 'SALUD1').\
withColumnRenamed('sum(SALUD2)', 'SALUD2').\
withColumnRenamed('sum(SALUD3)', 'SALUD3').\
withColumnRenamed('sum(SALUD4)', 'SALUD4').\
withColumnRenamed('sum(SALUD5)', 'SALUD5').\
withColumnRenamed('sum(SALUD6)', 'SALUD6')
salud.show(100, False)

+-------------------+---------+---------+---------+---------+---------+--------+
|ESTADO             |SALUD1   |SALUD2   |SALUD3   |SALUD4   |SALUD5   |SALUD6  |
+-------------------+---------+---------+---------+---------+---------+--------+
|Mexico             |9542506.0|4806894.0|5582779.0|744254.0 |2285966.0|108125.0|
|Jalisco            |5128892.0|2136824.0|3865818.0|178093.0 |808097.0 |15637.0 |
|Veracruz           |3547623.0|1413375.0|1797471.0|273741.0 |1173513.0|191960.0|
|Puebla             |3377488.0|1519386.0|1311769.0|180448.0 |1625086.0|24270.0 |
|Oaxaca             |1596748.0|769859.0 |428215.0 |209228.0 |840017.0 |41407.0 |
|CDMX               |6626503.0|2472027.0|3864930.0|1111978.0|1165768.0|86436.0 |
|Nuevo Leon         |4485750.0|1019247.0|3513884.0|153082.0 |443558.0 |24454.0 |
|Chihuahua          |2771221.0|478047.0 |1936334.0|145130.0 |437090.0 |6877.0  |
|Guanajuato         |3477520.0|923014.0 |1916622.0|238800.0 |1172732.0|32958.0 |
|Michoacan          |2097151

In [ ]:
salud.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/salud.csv")

### Situación Conyugal

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

situacion_conyugal = spark.sql("""select
                     CVEGEO,
                     if((isnull(SCONY1) or SCONY1 < 0 ), 0, SCONY1) AS SCONY1,
                     if((isnull(SCONY4) or SCONY4 < 0 ), 0, SCONY4) AS SCONY4,
                     if((isnull(SCONY7) or SCONY7 < 0 ), 0, SCONY7) AS SCONY7,
                     if((isnull(SCONY10) or SCONY10 < 0 ), 0, SCONY10) AS SCONY10
                     from censo""")

# Agrupación por Estados
situacion_conyugal = situacion_conyugal.withColumn("ESTADO", lit(None))
situacion_conyugal = situacion_conyugal.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
situacion_conyugal.show(100, False)

+----------------+------+------+------+-------+------+
|CVEGEO          |SCONY1|SCONY4|SCONY7|SCONY10|ESTADO|
+----------------+------+------+------+-------+------+
|1500100010127009|20.0  |16.0  |7.0   |0.0    |Mexico|
|1500100010127010|64.0  |71.0  |18.0  |3.0    |Mexico|
|1500100010127011|34.0  |51.0  |7.0   |3.0    |Mexico|
|1500100010127030|55.0  |62.0  |12.0  |0.0    |Mexico|
|1500100010127035|5.0   |6.0   |0.0   |0.0    |Mexico|
|1500100010269012|5.0   |3.0   |4.0   |0.0    |Mexico|
|1500100010288007|3.0   |16.0  |0.0   |3.0    |Mexico|
|1500100010324004|5.0   |18.0  |0.0   |4.0    |Mexico|
|1500100010339008|6.0   |16.0  |0.0   |0.0    |Mexico|
|1500100370305001|38.0  |38.0  |11.0  |0.0    |Mexico|
|1500100370305002|18.0  |40.0  |7.0   |0.0    |Mexico|
|1500100370305009|22.0  |29.0  |3.0   |6.0    |Mexico|
|1500100370305011|35.0  |69.0  |4.0   |8.0    |Mexico|
|1500100370305017|5.0   |8.0   |0.0   |0.0    |Mexico|
|1500100370305018|10.0  |17.0  |0.0   |0.0    |Mexico|
|150010037

In [ ]:
situacion_conyugal = situacion_conyugal.groupBy("ESTADO").sum(
    'SCONY1', 'SCONY4', 'SCONY7', 'SCONY10')
situacion_conyugal.show(100, False)

+-------------------+-----------+-----------+-----------+------------+
|ESTADO             |sum(SCONY1)|sum(SCONY4)|sum(SCONY7)|sum(SCONY10)|
+-------------------+-----------+-----------+-----------+------------+
|Mexico             |4131707.0  |6133352.0  |1450691.0  |373771.0    |
|Jalisco            |2156871.0  |2991306.0  |651381.0   |182751.0    |
|Veracruz           |1358666.0  |2136331.0  |551794.0   |113392.0    |
|Puebla             |1384350.0  |2038345.0  |452311.0   |155286.0    |
|Oaxaca             |628870.0   |1011051.0  |218042.0   |56672.0     |
|CDMX               |2989264.0  |3691768.0  |1146420.0  |144820.0    |
|Nuevo Leon         |1483077.0  |2480236.0  |473886.0   |168430.0    |
|Chihuahua          |905992.0   |1380755.0  |312991.0   |87513.0     |
|Guanajuato         |1239499.0  |1893476.0  |358220.0   |138700.0    |
|Michoacan          |866380.0   |1427349.0  |304090.0   |104891.0    |
|Guerrero           |530723.0   |852109.0   |212826.0   |59069.0     |
|Tamau

In [ ]:
situacion_conyugal.columns

['ESTADO', 'sum(SCONY1)', 'sum(SCONY4)', 'sum(SCONY7)', 'sum(SCONY10)']

In [ ]:
situacion_conyugal = situacion_conyugal.\
withColumnRenamed('sum(SCONY1)', 'SCONY1').\
withColumnRenamed('sum(SCONY4)', 'SCONY4').\
withColumnRenamed('sum(SCONY7)', 'SCONY7').\
withColumnRenamed('sum(SCONY10)', 'SCONY10')
situacion_conyugal.show(100, False)

+-------------------+---------+---------+---------+--------+
|ESTADO             |SCONY1   |SCONY4   |SCONY7   |SCONY10 |
+-------------------+---------+---------+---------+--------+
|Mexico             |4131707.0|6133352.0|1450691.0|373771.0|
|Jalisco            |2156871.0|2991306.0|651381.0 |182751.0|
|Veracruz           |1358666.0|2136331.0|551794.0 |113392.0|
|Puebla             |1384350.0|2038345.0|452311.0 |155286.0|
|Oaxaca             |628870.0 |1011051.0|218042.0 |56672.0 |
|CDMX               |2989264.0|3691768.0|1146420.0|144820.0|
|Nuevo Leon         |1483077.0|2480236.0|473886.0 |168430.0|
|Chihuahua          |905992.0 |1380755.0|312991.0 |87513.0 |
|Guanajuato         |1239499.0|1893476.0|358220.0 |138700.0|
|Michoacan          |866380.0 |1427349.0|304090.0 |104891.0|
|Guerrero           |530723.0 |852109.0 |212826.0 |59069.0 |
|Tamaulipas         |841084.0 |1382692.0|304048.0 |87358.0 |
|Sonora             |727856.0 |1076485.0|252691.0 |57217.0 |
|Chiapas            |680

In [ ]:
situacion_conyugal.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/situacion_conyugal.csv")

### Religión

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

religion = spark.sql("""select 
                     CVEGEO,
                     if((isnull(RELIG1) or RELIG1 < 0 ), 0, RELIG1) AS RELIG1,
                     if((isnull(RELIG2) or RELIG2 < 0 ), 0, RELIG2) AS RELIG2,
                     if((isnull(RELIG3) or RELIG3 < 0 ), 0, RELIG3) AS RELIG3,
                     if((isnull(RELIG4) or RELIG4 < 0 ), 0, RELIG4) AS RELIG4
                     from censo""")

# Agrupación por Estados
religion = religion.withColumn("ESTADO", lit(None))
religion = religion.withColumn("ESTADO", when(col('CVEGEO').like("01%"), "Aguascalientes")
                                 .when(col('CVEGEO').like("02%"), "Baja California")
                                 .when(col('CVEGEO').like("03%"), "Baja California Sur")
                                 .when(col('CVEGEO').like("04%"), "Campeche")
                                 .when(col('CVEGEO').like("05%"), "Coahuila")
                                 .when(col('CVEGEO').like("06%"), "Colima")
                                 .when(col('CVEGEO').like("07%"), "Chiapas")
                                 .when(col('CVEGEO').like("08%"), "Chihuahua")
                                 .when(col('CVEGEO').like("09%"), "CDMX")
                                 .when(col('CVEGEO').like("10%"), "Durango")
                                 .when(col('CVEGEO').like("11%"), "Guanajuato")
                                 .when(col('CVEGEO').like("12%"), "Guerrero")
                                 .when(col('CVEGEO').like("13%"), "Hidalgo")
                                 .when(col('CVEGEO').like("14%"), "Jalisco")
                                 .when(col('CVEGEO').like("15%"), "Mexico")
                                 .when(col('CVEGEO').like("16%"), "Michoacan")
                                 .when(col('CVEGEO').like("17%"), "Morelos")
                                 .when(col('CVEGEO').like("18%"), "Nayarit")
                                 .when(col('CVEGEO').like("19%"), "Nuevo Leon")
                                 .when(col('CVEGEO').like("20%"), "Oaxaca")
                                 .when(col('CVEGEO').like("21%"), "Puebla")
                                 .when(col('CVEGEO').like("22%"), "Queretaro")
                                 .when(col('CVEGEO').like("23%"), "Quintana Roo")
                                 .when(col('CVEGEO').like("24%"), "San Luis Potosi")
                                 .when(col('CVEGEO').like("25%"), "Sinaloa")
                                 .when(col('CVEGEO').like("26%"), "Sonora")
                                 .when(col('CVEGEO').like("27%"), "Tabasco")
                                 .when(col('CVEGEO').like("28%"), "Tamaulipas")
                                 .when(col('CVEGEO').like("29%"), "Tlaxcala")
                                 .when(col('CVEGEO').like("30%"), "Veracruz")
                                 .when(col('CVEGEO').like("31%"), "Yucatan")
                                 .when(col('CVEGEO').like("32%"), "Zacatecas"))

In [ ]:
religion.show(100, False)

+----------------+------+------+------+------+------+
|CVEGEO          |RELIG1|RELIG2|RELIG3|RELIG4|ESTADO|
+----------------+------+------+------+------+------+
|1500100010127009|47.0  |0.0   |0.0   |0.0   |Mexico|
|1500100010127010|179.0 |0.0   |0.0   |0.0   |Mexico|
|1500100010127011|108.0 |0.0   |0.0   |0.0   |Mexico|
|1500100010127030|142.0 |8.0   |0.0   |9.0   |Mexico|
|1500100010127035|14.0  |0.0   |0.0   |0.0   |Mexico|
|1500100010269012|13.0  |0.0   |0.0   |0.0   |Mexico|
|1500100010288007|29.0  |0.0   |0.0   |0.0   |Mexico|
|1500100010324004|12.0  |5.0   |0.0   |15.0  |Mexico|
|1500100010339008|34.0  |0.0   |0.0   |0.0   |Mexico|
|1500100370305001|69.0  |37.0  |0.0   |0.0   |Mexico|
|1500100370305002|70.0  |18.0  |0.0   |6.0   |Mexico|
|1500100370305009|30.0  |19.0  |0.0   |20.0  |Mexico|
|1500100370305011|140.0 |4.0   |0.0   |0.0   |Mexico|
|1500100370305017|19.0  |0.0   |0.0   |0.0   |Mexico|
|1500100370305018|13.0  |7.0   |0.0   |11.0  |Mexico|
|1500100370305033|17.0  |0.0

In [ ]:
religion = religion.groupBy("ESTADO").sum(
    'RELIG1', 'RELIG2', 'RELIG3', 'RELIG4')
religion.show(100, False)

+-------------------+-----------+-----------+-----------+-----------+
|ESTADO             |sum(RELIG1)|sum(RELIG2)|sum(RELIG3)|sum(RELIG4)|
+-------------------+-----------+-----------+-----------+-----------+
|Mexico             |1.1129406E7|1140219.0  |46240.0    |1972226.0  |
|Jalisco            |6509074.0  |344700.0   |4726.0     |366066.0   |
|Veracruz           |3664619.0  |744446.0   |3258.0     |518432.0   |
|Puebla             |4118675.0  |399385.0   |5672.0     |351845.0   |
|Oaxaca             |1793821.0  |347543.0   |1176.0     |208831.0   |
|CDMX               |6919670.0  |649641.0   |42946.0    |1452401.0  |
|Nuevo Leon         |4304090.0  |650018.0   |2392.0     |526652.0   |
|Chihuahua          |2378366.0  |388804.0   |1262.0     |462141.0   |
|Guanajuato         |3959199.0  |193104.0   |2409.0     |223567.0   |
|Michoacan          |2922400.0  |178653.0   |1317.0     |199444.0   |
|Guerrero           |1584762.0  |226008.0   |1032.0     |224875.0   |
|Tamaulipas         

In [ ]:
religion.columns

['ESTADO', 'sum(RELIG1)', 'sum(RELIG2)', 'sum(RELIG3)', 'sum(RELIG4)']

In [ ]:
religion = religion.\
withColumnRenamed('sum(RELIG1)', 'RELIG1').\
withColumnRenamed('sum(RELIG2)', 'RELIG2').\
withColumnRenamed('sum(RELIG3)', 'RELIG3').\
withColumnRenamed('sum(RELIG4)', 'RELIG4')
religion.show(100, False)

+-------------------+-----------+---------+-------+---------+
|ESTADO             |RELIG1     |RELIG2   |RELIG3 |RELIG4   |
+-------------------+-----------+---------+-------+---------+
|Mexico             |1.1129406E7|1140219.0|46240.0|1972226.0|
|Jalisco            |6509074.0  |344700.0 |4726.0 |366066.0 |
|Veracruz           |3664619.0  |744446.0 |3258.0 |518432.0 |
|Puebla             |4118675.0  |399385.0 |5672.0 |351845.0 |
|Oaxaca             |1793821.0  |347543.0 |1176.0 |208831.0 |
|CDMX               |6919670.0  |649641.0 |42946.0|1452401.0|
|Nuevo Leon         |4304090.0  |650018.0 |2392.0 |526652.0 |
|Chihuahua          |2378366.0  |388804.0 |1262.0 |462141.0 |
|Guanajuato         |3959199.0  |193104.0 |2409.0 |223567.0 |
|Michoacan          |2922400.0  |178653.0 |1317.0 |199444.0 |
|Guerrero           |1584762.0  |226008.0 |1032.0 |224875.0 |
|Tamaulipas         |2147041.0  |549863.0 |1048.0 |448964.0 |
|Sonora             |1971048.0  |280016.0 |486.0  |301792.0 |
|Chiapas

In [ ]:
religion.toPandas().to_csv("/content/gdrive/Shareddrives/Reto INEGI/Base de Datos/religion.csv")